In [1]:
from keras import Input, Model
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2

# from spektral.layers import GraphConv
# GRaphConv is deprecated, use GCNConv or GCSConv instead
from spektral.utils.sparse import sp_matrix_to_sp_tensor
from spektral.utils import normalized_laplacian
from spektral.layers import GCSConv  # as GraphConv
from spektral.layers import GINConv # as GraphConv
from spektral.layers import GCNConv  # as GraphConv

from spektral.utils.convolution import gcn_filter  # For GCNConv
from spektral.utils.convolution import normalized_adjacency  # For GCSConv

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [2]:
# Parameters
l2_reg = 5e-4  # Regularization rate for l2
learning_rate = 1e-3  # Learning rate for SGD
batch_size = 32  # Batch size
epochs = 5  # Number of training epochs
es_patience = 200  # Patience fot early stopping

In [3]:
# Load one adjacency matrix and its features
# teste
# load 10 npz files from the folder angry_adj

import pathlib
import os
import numpy as np
import scipy.sparse

actual_path = pathlib.Path().absolute()

# subindo um nível

path = actual_path.parent


# load the npz file
# the npz file is a sparse matrix

adj_angry_path = path / 'angry_adj'
print(adj_angry_path)
for file in os.listdir(adj_angry_path)[:1]:
    file_path = adj_angry_path / file
    sparse_matrix = scipy.sparse.load_npz(file_path)
    sparse_matrix = sparse_matrix.todense()

# as all the matrices are equal, we can use the first one is equal to all the others

angry_path = path / 'surprised_dist'
disgusted_path = path / 'disgusted_dist'
happy_path = path / 'happy_dist'
neutral_path = path / 'neutral_dist'
sad_path = path / 'sad_dist'
surprised_path = path / 'surprised_dist'

path_list = [angry_path, disgusted_path, happy_path, neutral_path, sad_path, surprised_path]

# load the json files of the distances (features)

import json

# load the json files
dists_surprised = []
dists_disgusted = []
dists_happy = []
dists_neutral = []
dists_sad = []
dists_angry = []

dists_list = [dists_angry, dists_disgusted, dists_happy, dists_neutral, dists_sad, dists_surprised]

for path in path_list:
    for file in os.listdir(path):
        file_path = path / file
        with open(file_path, 'r') as f:
            data = json.load(f)
            dists_list[path_list.index(path)].append(data)

print(len(dists_angry))


c:\Users\rodri\Documents\Rodrigo\Insper\SextoSemestre\Facial-Emotion-Classification-Graph_Fork\angry_adj
8024


In [4]:
sparse_matrix.shape

(468, 468)

In [5]:
# dimension of the target
n_out = 6

N = (sparse_matrix.shape[0])  # Number of nodes in the graph
print(N)

468


In [6]:
# dimensions of the features
F = 1

In [7]:
y = np.arange(n_out)  # Target labels
print(y)


[0 1 2 3 4 5]


In [8]:
# 
# extract all nodes from the adjacency matrix

def extract_nodes(adj_matrix):
    nodes = []
    for i in range(adj_matrix.shape[0]):
        nodes.append(i)
    return nodes

nodes = extract_nodes(sparse_matrix)
print(nodes)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [43]:
# create a mtrix that is the adjancency matrix multiplied by the features
# the features are the distances between the nodes
# each line of the matrix is a node and the columns are the distances between the node and the other nodes
# para cada ponto = 1 na matriz de adjacencia, multiplica pela distancia entre os pontos
# o resultado é a matriz de features

def create_features_matrix(adj_matrix, dists_list):
    features_matrix = np.zeros((adj_matrix.shape[0], adj_matrix.shape[0]))
    for i in range(adj_matrix.shape[0]):
        for j in range(adj_matrix.shape[0]):
            if adj_matrix[i, j] == 1:
                #print(dists_list[i])
                features_matrix[i, j] = dists_list[i]
    return features_matrix


features_matrix = create_features_matrix(sparse_matrix, dists_list[0][0])
print(features_matrix.shape)
print(features_matrix)

(468, 468)
[[ 0.         89.63977946  0.         ...  0.          0.
   0.        ]
 [41.13040241  0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [ ]:
# for each distance matrix, create a features matrix
features_matrix_list = []
for dists in dists_list:
    print(dists)
    features_matrix = create_features_matrix(sparse_matrix, dists[0])
    features_matrix_list.append(features_matrix)

print(len(features_matrix_list))

IndexError: list index out of range

https://graphneural.network/layers/convolution/#gcsconv

In [9]:
# Prepare data
fltr = normalized_adjacency(sparse_matrix)

# Model using GCSConv
X_in = Input(shape=(N, F))
# Pass A as a fixed tensor, otherwise Keras will complain about inputs of
# different rank.
A_in = Input(tensor=sp_matrix_to_sp_tensor(fltr))

graph_conv_1 = GCSConv(
    32, activation="elu", kernel_regularizer=l2(l2_reg), use_bias=True
)([X_in, A_in])
graph_conv_2 = GCSConv(
    32, activation="elu", kernel_regularizer=l2(l2_reg), use_bias=True
)([graph_conv_1, A_in])
flatten = Flatten()(graph_conv_2)
fc = Dense(512, activation="relu")(flatten)
output = Dense(n_out, activation="softmax")(fc)

# Build model
model = Model(inputs=[X_in, A_in], outputs=output)
optimizer = Adam(lr=learning_rate)
model.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["acc"]
)
model.summary()

c:\Users\rodri\anaconda3\envs\mirror\Lib\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 468, 1)]             0         []                            
                                                                                                  
 input_2 (InputLayer)        [(468, 468)]                 0         []                            
                                                                                                  
 gcs_conv (GCSConv)          (None, 468, 32)              96        ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 gcs_conv_1 (GCSConv)        (None, 468, 32)              2080      ['gcs_conv[0][0]',        

In [ ]:
# train the GSCConv model
model.fit(
    [nodes, fltr],
    y,
    batch_size=batch_size,
    validation_split=0.1,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=es_patience, restore_best_weights=True)],
)

https://graphneural.network/layers/convolution/#gcnconv

In [ ]:
# Prepare data
fltr = gcn_filter(adj)
# Model using GCNConv
X_in = Input(shape=(N, F))
# Pass A as a fixed tensor, otherwise Keras will complain about inputs of
# different rank.
A_in = Input(tensor=sp_matrix_to_sp_tensor(fltr))

graph_conv_1 = GCNConv(
    32, activation="elu", kernel_regularizer=l2(l2_reg), use_bias=True
)([X_in, A_in])
graph_conv_2 = GCNConv(
    32, activation="elu", kernel_regularizer=l2(l2_reg), use_bias=True
)([graph_conv_1, A_in])
# removed the flatten layer because it is not necessary for GCNConv
fc = Dense(512, activation="relu")(graph_conv_2)
output = Dense(n_out, activation="softmax")(fc)


# Build model
model = Model(inputs=[X_in, A_in], outputs=output)
optimizer = Adam(lr=learning_rate)
model.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["acc"]
)
model.summary()

In [ ]:
# train the GCNConv model

model.fit(
    [nodes, fltr],
    y,
    batch_size=batch_size,
    validation_split=0.1,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=es_patience, restore_best_weights=True)],
)